In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/large_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model.mlp import MLP
from model_config import MLP_Models, LSTM_Models, GRU_Models
from utils.benchmark import Benchmark

In [2]:
conf = Config()
model_conf = LSTM_Models()
model = model_conf.get_model(model_conf.lstm_4)

In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "mlp")

mlp_dataset = NetworkDataset(X_test, y_test)
mlp_loader = DataLoader(mlp_dataset, conf.batch_size)

X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "lstm")

rnn_dataset = NetworkDataset(X_test, y_test)
rnn_loader = DataLoader(rnn_dataset, conf.batch_size)

In [ ]:
# set parameters
loader = mlp_loader if isinstance(model.model, MLP) else rnn_loader
name = model_conf.lstm_4["name"]
result_path = os.path.join(conf.benchmark_host, "large_model", name + ".txt")

# run benchmark
benchmark = Benchmark(model, loader, conf.batch_size, name, result_path)
benchmark()

# print and save result
benchmark.print_result()
benchmark.save()

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/large_model/lstm_4.pth!


[W1221 12:11:09.222433668 CPUAllocator.cpp:245] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event
ERROR:2025-12-21 12:11:18 798613:798613 DeviceProperties.cpp:47] gpuGetDeviceCount failed with code 35


Benchmark - lstm_4 model:

Memory usage (MB):
Avg memory usage: 399.219MB
Peak memory usage: 5183.784MB

Model inference latency on one batch (batch size = 512):
Avg latency: 925.895ms
Min latency: 865.377ms
Max latency: 1042.870ms

Model inference throughput (batch size = 512):
Throughput: 578.71 samples/sec

Model inference CPU usage (number of logical cores) during runtime:
CPU runtime: 46.54 seconds
Average CPU usage: 35.63/96 cores

Model (lstm_4) inference accuracy (%):
Accuracy: 97.07%



